In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [2]:
# Load R02_Data.csv
data = pd.read_csv("./R02_Data.csv")
# Display basic information about the dataset
print(data.info())
print(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 974836 entries, 0 to 974835
Data columns (total 10 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   _time                     974836 non-null  object 
 1   I_R02_Gripper_Load        974836 non-null  int64  
 2   I_R02_Gripper_Pot         974836 non-null  int64  
 3   M_R02_BJointAngle_Degree  974836 non-null  float64
 4   M_R02_LJointAngle_Degree  974836 non-null  float64
 5   M_R02_RJointAngle_Degree  974836 non-null  float64
 6   M_R02_SJointAngle_Degree  974836 non-null  float64
 7   M_R02_TJointAngle_Degree  974836 non-null  float64
 8   M_R02_UJointAngle_Degree  974836 non-null  float64
 9   Description               492728 non-null  object 
dtypes: float64(6), int64(2), object(2)
memory usage: 74.4+ MB
None
                      _time  I_R02_Gripper_Load  I_R02_Gripper_Pot  \
0  2023-12-11T13:04:12.348Z                1299              11695   
1  2023-1

C:\Users\Alireza Vaezi\AppData\Local\Temp\ipykernel_47288\698973302.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("./R02_Data.csv")


In [3]:
# Key columns for analysis
gripper_load = data["I_R02_Gripper_Load"]
gripper_pot = data["I_R02_Gripper_Pot"]
# Descriptive statistics
print("Gripper Load Statistics:")
print(gripper_load.describe())
print("\nGripper Potentiometer Statistics:")
print(gripper_pot.describe())
# Plot time series for both variables
plt.figure(figsize=(14, 6))
plt.plot(data["_time"], gripper_load, label="Gripper Load", alpha=0.8)
plt.plot(data["_time"], gripper_pot, label="Gripper Potentiometer", alpha=0.8, color="orange")
plt.title("Time Series of Gripper Load and Potentiometer")
plt.xlabel("Time")
plt.ylabel("Values")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

Gripper Load Statistics:
count    974836.00000
mean       1306.58103
std          15.91377
min        1147.00000
25%        1296.00000
50%        1307.00000
75%        1317.00000
max        1474.00000
Name: I_R02_Gripper_Load, dtype: float64

Gripper Potentiometer Statistics:
count    974836.000000
mean      10895.532439
std        2123.800414
min        2001.000000
25%       11601.000000
50%       11642.000000
75%       11680.000000
max       11998.000000
Name: I_R02_Gripper_Pot, dtype: float64


KeyboardInterrupt: 

In [ ]:
# SPC metrics
mean_load = gripper_load.mean()
std_load = gripper_load.std()
ucl_load = mean_load + 3 * std_load
lcl_load = mean_load - 3 * std_load
# SPC chart
plt.figure(figsize=(14, 6))
plt.plot(data["_time"], gripper_load, label="Gripper Load", alpha=0.8)
plt.axhline(mean_load, color="green", linestyle="--", label="Mean")
plt.axhline(ucl_load, color="red", linestyle="--", label="UCL")
plt.axhline(lcl_load, color="blue", linestyle="--", label="LCL")
plt.title("SPC Chart for Gripper Load")
plt.xlabel("Time")
plt.ylabel("Load")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
# Summarize SPC results
print(f"Mean: {mean_load}, Standard Deviation: {std_load}, UCL: {ucl_load}, LCL: {lcl_load}")

KeyboardInterrupt: 

In [ ]:
# Refined SPC metrics
mean_pot = gripper_pot.mean()
std_pot = gripper_pot.std()
ucl_pot = min(gripper_pot.max(), mean_pot + 3 * std_pot)
lcl_pot = max(gripper_pot.min(), mean_pot - 3 * std_pot)
# SPC chart for Potentiometer
plt.figure(figsize=(14, 6))
plt.plot(data["_time"], gripper_pot, label="Potentiometer", alpha=0.8, color="orange")
plt.axhline(mean_pot, color="green", linestyle="--", label="Mean")
plt.axhline(ucl_pot, color="red", linestyle="--", label="UCL")
plt.axhline(lcl_pot, color="blue", linestyle="--", label="LCL")
plt.title("SPC Chart for Potentiometer with Refined Thresholds")
plt.xlabel("Time")
plt.ylabel("Potentiometer Value")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
# Summarize SPC results
print(f"Mean: {mean_pot}, Standard Deviation: {std_pot}, UCL: {ucl_pot}, LCL: {lcl_pot}")

In [ ]:
# Combine key columns for clustering
clustering_data = data[["I_R02_Gripper_Load", "I_R02_Gripper_Pot"]].dropna()
# Apply K-Means clustering
kmeans = KMeans(n_clusters=3, random_state=42)
clustering_data["Cluster"] = kmeans.fit_predict(clustering_data)
# Scatter plot of clusters
plt.figure(figsize=(14, 6))
plt.scatter(clustering_data["I_R02_Gripper_Load"], clustering_data["I_R02_Gripper_Pot"], c=clustering_data["Cluster"], cmap="viridis", alpha=0.6)
plt.title("Clustering Results: Gripper Load vs Potentiometer")
plt.xlabel("Gripper Load")
plt.ylabel("Potentiometer")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Example table summarizing SPC results
spc_results = pd.DataFrame({"Metric": ["Mean", "Standard Deviation", "UCL (3σ)", "LCL (3σ)"], "Gripper Load": [mean_load, std_load, ucl_load, lcl_load], "Gripper Potentiometer": [mean_pot, std_pot, ucl_pot, lcl_pot]})
print(spc_results)

In [ ]:
# Adaptive thresholds for Gripper Load
adaptive_ucl_load = mean_load + 2 * std_load
adaptive_lcl_load = mean_load - 2 * std_load
# Identify anomalies
load_anomalies = gripper_load_data[(gripper_load_data > adaptive_ucl_load) | (gripper_load_data < adaptive_lcl_load)]
# Plot with anomalies
plt.figure(figsize=(14, 6))
plt.plot(data["_time"], gripper_load_data, label="Gripper Load Values", alpha=0.8)
plt.axhline(mean_load, color="green", linestyle="--", label="Mean")
plt.axhline(adaptive_ucl_load, color="orange", linestyle="--", label="Adaptive UCL (2σ)")
plt.axhline(adaptive_lcl_load, color="orange", linestyle="--", label="Adaptive LCL (2σ)")
plt.scatter(load_anomalies.index, load_anomalies, color="red", label="Anomalies", alpha=0.6)
plt.title("SPC Chart for Gripper Load with Adaptive Thresholds")
plt.xlabel("Time")
plt.ylabel("Gripper Load Value")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
# Summarize adaptive threshold anomalies
print(f"Total anomalies detected with adaptive thresholds: {len(load_anomalies)}")

In [ ]:
# Calculate slopes (rate of change)
gripper_load_slope = gripper_load_data.diff()
gripper_pot_slope = gripper_pot.diff()
# Detect peaks and dips
from scipy.signal import find_peaks

load_peaks, _ = find_peaks(gripper_load_data, height=mean_load + std_load)
load_dips, _ = find_peaks(-gripper_load_data, height=-(mean_load - std_load))
# Plot slopes and peaks
plt.figure(figsize=(14, 6))
plt.plot(data["_time"], gripper_load_data, label="Gripper Load Values", alpha=0.8)
plt.scatter(load_peaks, gripper_load_data[load_peaks], color="red", label="Peaks")
plt.scatter(load_dips, gripper_load_data[load_dips], color="blue", label="Dips")
plt.title("Peaks and Dips in Gripper Load")
plt.xlabel("Time")
plt.ylabel("Gripper Load")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
# Summarize findings
print(f"Total Peaks: {len(load_peaks)}, Total Dips: {len(load_dips)}")

In [ ]:
# Process Capability Analysis for Gripper Load
usl_load, lsl_load = ucl_load, lcl_load  # Using control limits as spec limits
cp_load = (usl_load - lsl_load) / (6 * std_load)
cpk_load = min((usl_load - mean_load) / (3 * std_load), (mean_load - lsl_load) / (3 * std_load))
# Process Capability Analysis for Gripper Potentiometer
usl_pot, lsl_pot = ucl_pot, lcl_pot
cp_pot = (usl_pot - lsl_pot) / (6 * std_pot)
cpk_pot = min((usl_pot - mean_pot) / (3 * std_pot), (mean_pot - lsl_pot) / (3 * std_pot))
# Summarize results
print(f"Gripper Load - Cp: {cp_load}, Cpk: {cpk_load}")
print(f"Gripper Potentiometer - Cp: {cp_pot}, Cpk: {cpk_pot}")

In [ ]:
# Compile SPC metrics into a DataFrame
spc_results = pd.DataFrame({"Metric": ["Mean", "Standard Deviation", "UCL (3σ)", "LCL (3σ)", "Cp", "Cpk"], "Gripper Load": [mean_load, std_load, ucl_load, lcl_load, cp_load, cpk_load], "Gripper Potentiometer": [mean_pot, std_pot, ucl_pot, lcl_pot, cp_pot, cpk_pot]})
# Save results to CSV
spc_results.to_csv("SPC_Results.csv", index=False)
print("SPC results saved to SPC_Results.csv.")

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Plot ACF and PACF for Gripper Load
plt.figure(figsize=(12, 6))
plot_acf(gripper_load_data, lags=50, title="ACF for Gripper Load")
plt.show()
plt.figure(figsize=(12, 6))
plot_pacf(gripper_load_data, lags=50, title="PACF for Gripper Load")
plt.show()

In [ ]:
# Combine and normalize data
from sklearn.preprocessing import StandardScaler

features = ["I_R02_Gripper_Load", "I_R02_Gripper_Pot", "M_R02_BJointAngle_Degree", "M_R02_LJointAngle_Degree"]
multi_modal_data = data[features].dropna()
scaler = StandardScaler()
normalized_data = scaler.fit_transform(multi_modal_data)
# Perform clustering on normalized data
kmeans_multi = KMeans(n_clusters=3, random_state=42)
multi_modal_data["Cluster"] = kmeans_multi.fit_predict(normalized_data)
# Visualize clusters in 2D (using first two principal components)
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca_result = pca.fit_transform(normalized_data)
plt.figure(figsize=(14, 6))
plt.scatter(pca_result[:, 0], pca_result[:, 1], c=multi_modal_data["Cluster"], cmap="viridis", alpha=0.6)
plt.title("Multi-Modal Clustering (PCA)")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.grid(True)
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Label anomalies as 1 and normal points as 0 (based on SPC limits)
data["Anomaly_Load"] = ((data["I_R02_Gripper_Load"] > ucl_load) | (data["I_R02_Gripper_Load"] < lcl_load)).astype(int)
# Features and target
X = multi_modal_data[features]
y = data["Anomaly_Load"]
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Train Random Forest
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
# Evaluate model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
import time

# Simulate real-time data streaming
for i in range(0, len(data), 10):  # Stream data in chunks of 10
    chunk = data.iloc[i : i + 10]
    print(f"Processing chunk {i}-{i+10}...")
    # Detect anomalies in the chunk
    chunk_anomalies = chunk[(chunk["I_R02_Gripper_Load"] > ucl_load) | (chunk["I_R02_Gripper_Load"] < lcl_load)]
    if not chunk_anomalies.empty:
        print("Anomalies detected:")
        print(chunk_anomalies)
    time.sleep(1)  # Simulate a delay for real-time streaming

In [ ]:
import seaborn as sns

# Filter data for anomalies
anomalous_data = data[(data["I_R02_Gripper_Load"] > ucl_load) | (data["I_R02_Gripper_Load"] < lcl_load)]
# Correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(anomalous_data[features].corr(), annot=True, cmap="coolwarm")
plt.title("Correlation Heatmap for Anomalous Data")
plt.show()

In [ ]:
import plotly.express as px

# Interactive SPC chart for Gripper Load
fig = px.line(data, x="_time", y="I_R02_Gripper_Load", title="Interactive SPC Chart for Gripper Load")
fig.add_hline(y=mean_load, line_dash="dash", annotation_text="Mean")
fig.add_hline(y=ucl_load, line_dash="dash", line_color="red", annotation_text="UCL")
fig.add_hline(y=lcl_load, line_dash="dash", line_color="blue", annotation_text="LCL")
fig.show()